# Segmenting and Clustering Neighborhoods in Riyadh-KSA

## 1-Data ingestion

In course we used beautifulSoup for web scraping the neighborhoods. For our project we will use different web scraping tool which is Selenium for different reasons:

-BeautifulSoup is good for static content, but it is limited in dynamic websites

-Selenium is very powerfull tool to scrape dynamic websites like ours, we need to interact with the website content and make some choices in some components in the website

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [184]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
import time

we need to download the chrome driver from this url https://chromedriver.storage.googleapis.com/index.html?path=74.0.3729.6/ and unzip the content in "C:\chromedriver_win32\"

In [159]:
chrome_path = r"C:\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

Now let's open our website in chrome driver browser

In [160]:
driver.get("http://maps.address.gov.sa/")

The default language of the site is Arabic. In order to change the language we will simulate a click the change language hyperlink

We need to inspect this element and get its id "LanguageArEn".  we will query the DOM using find_element_by_id function and find it and click it

In [161]:
Changelanguage =driver.find_element_by_id("LanguageArEn")
Changelanguage.click()

Click on Advanced Search hyperlink

In [162]:
OpenAdvancedSearch =driver.find_element_by_id("hrefOpenAdvancedSearch")
OpenAdvancedSearch.click()

Click the "Search by address" tab

In [163]:
AdvancedByAddressOption =driver.find_element_by_id("hrefAdvancedByAddressOption")
AdvancedByAddressOption.click()

In order to get the list of Riyadh Neighborhood we need to :

-choose the right Region from the Region dropdownList : Riyadh

-Choose the right city from the City dropdownList : RIYADH

#### Pick Riyadh Region from the dropdownList

In [86]:
wait = WebDriverWait(driver, 10)
try:
    #ddlRegion = driver.find_element_by_xpath("""//*[@id="ddlRegion_chosen"]""")
    #ddlRegion = driver.find_element_by_id("ddlRegion")
    #ddlRegion = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="ddlRegion"]""")))
    #ddlRegion.location_once_scrolled_into_view
    driver.execute_script("window.scrollTo(0, 0);")
    ddlRegion = wait.until(EC.visibility_of_element_located((By.XPATH,"""//*[@id="ddlRegion"]""")))
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("arguments[0].scrollIntoView();", ddlRegion)
    ddlRegion
   
    #Regionselect = Select(ddlRegion)
    #Regionselect.select_by_index(13)
    #myselect.select_by_visible_text("Riyadh")
except TimeoutException:
    print(TimeoutException)

<class 'selenium.common.exceptions.TimeoutException'>


In [164]:
ddlRegion = driver.find_element_by_id("ddlRegion")
Regionselect = Select(ddlRegion)
#Regionselect.select_by_index(1)
Regionselect.select_by_visible_text("Riyadh")

ElementNotVisibleException: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=75.0.3770.80)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)


As you can see the above error: element not interactable Element is not currently visible and may not be manipulated

I tried different solutions for this but it is not working

So i choosed other DOM elements to interact with

In [165]:
ddlRegion = driver.find_element_by_id("ddlRegion")
driver.find_element_by_xpath("""//*[@id="ddlRegion_chosen"]/a""").click()
txtRegion = driver.find_element_by_xpath("""//*[@id="ddlRegion_chosen"]/div/div/input""")
txtRegion.send_keys("Riyadh")
txtRegion.send_keys(u'\ue007')

In [167]:
driver.find_element_by_xpath("""//*[@id="ddlCity_chosen"]/a""").click()
txtCity = driver.find_element_by_xpath("""//*[@id="ddlCity_chosen"]/div/div/input""")
txtCity.send_keys("RIYADH")
txtCity.send_keys(u'\ue007')
ddlCity = driver.find_element_by_id("ddlCity")
Cityselect = Select(ddlCity)
option = Cityselect.first_selected_option
CityID = option.get_attribute("id")
print('The Riyadh CityID is : {} .'.format(CityID)) 

The Riyadh CityID is : 3 .


To fetch the neighborhoods We will call the REST Service http://maps.address.gov.sa/Home/GetDistricts with CityID parameter

In [168]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd
url = 'http://maps.address.gov.sa/Home/GetDistricts?cityID=' + CityID;
response = requests.post(url);
resp = response.text;
data = json.loads(resp);
df = json_normalize(data);
#df1 = pd.DataFrame(df)
df['XCoord'] = df.XCoord.astype(float)
df['YCoord'] = df.YCoord.astype(float)
df.head(10)
#df.dtypes

,ArabicName,EnglishName,OBJECTID,XCoord,YCoord,fkCityID,pkDistrictID
0,حي العمل,Al Amal Dist.,8087,$46.72269,$24.64592,3,10100003001
1,حي النموذجية,Al Namudhajiyah Dist.,8046,$46.69462,$24.65685,3,10100003002
2,حي الجرادية,Al Jarradiyah Dist.,8003,$46.69777,$24.61703,3,10100003003
3,حي الصناعية,Al Sinaiyah Dist.,7996,$46.74722,$24.64239,3,10100003004
4,حي منفوحة الجديدة,Manfuha Al Jadidah Dist.,8002,$46.71846,$24.61277,3,10100003005
5,حي الفاخرية,Al Fakhiriyah Dist.,7990,$46.68283,$24.64162,3,10100003006
6,حي الديرة,Al Dirah Dist.,7993,$46.71010,$24.63230,3,10100003007
7,حي ام الحمام الشرقي,East Umm Al Hamam Dist.,8041,$46.65804,$24.68646,3,10100003008
8,حي الشرفية,Al Sharafiyah Dist.,8076,$46.66883,$24.65865,3,10100003009
9,حي الهدا,Al Hada Dist.,8144,$46.64068,$24.66074,3,10100003010


In [169]:
print('Therea are {} neighborhoods in  Riyadh City'.format(df1.shape[0]))

Therea are 192 neighborhoods in  Riyadh City


we will rename columns and remove some columns

In [170]:
from decimal import Decimal
filtered_columns = ['EnglishName', 'XCoord', 'YCoord']
neighborhoods =df.loc[:, filtered_columns]
neighborhoods.columns = ['Neighborhood', 'Longitude', 'Latitude']
#pd.options.display.float_format = '${:,.5f}'.format
#neighborhoods.round(5)
#neighborhoods.head()
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].astype(str)

#neighborhoods['Longitude'] = neighborhoods['Longitude'].str.replace('$', '')
#neighborhoods.Longitude = [x.strip('$') for x in neighborhoods.Longitude]

neighborhoods['Longitude'] = neighborhoods.Longitude.astype(str)
#neighborhoods['Longitude'] = neighborhoods.Longitude.astype(float)
#neighborhoods['Longitude'] = pd.to_numeric(neighborhoods.Longitude, errors='coerce')
#neighborhoods.Longitude.apply(lambda x: x.replace('$',''))

#neighborhoods['Latitude'] = neighborhoods.Latitude.astype(float)
#neighborhoods['Latitude'] = pd.to_numeric(neighborhoods['Latitude'], errors='coerce')
neighborhoods = neighborhoods.astype(dtype= {"Neighborhood":"str",
        "Longitude":"float64","Latitude":"float64"})
neighborhoods.head(10)#dtypes#head(10)#

,Neighborhood,Longitude,Latitude
0,Al Amal Dist.,$46.72269,$24.64592
1,Al Namudhajiyah Dist.,$46.69462,$24.65685
2,Al Jarradiyah Dist.,$46.69777,$24.61703
3,Al Sinaiyah Dist.,$46.74722,$24.64239
4,Manfuha Al Jadidah Dist.,$46.71846,$24.61277
5,Al Fakhiriyah Dist.,$46.68283,$24.64162
6,Al Dirah Dist.,$46.71010,$24.63230
7,East Umm Al Hamam Dist.,$46.65804,$24.68646
8,Al Sharafiyah Dist.,$46.66883,$24.65865
9,Al Hada Dist.,$46.64068,$24.66074


Let's export the results to csv file

In [171]:
export_csv = neighborhoods.to_csv (r'D:\Riyadh_Neighborhoods.csv', index = None, header=True,encoding='utf-8')

## 2-Data visualization

In [7]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chaira-h\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.1.0               |           py37_0         721 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.6.14               |           py37_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  altair             conda



==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda


'ET' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chaira-h\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0




geographiclib-1.49   | 32 KB     |            |   0% 
geographiclib-1.49   | 32 KB     | ###7       |  37% 
geographiclib-1.49   | 32 KB     | ########## | 100% 

geopy-1.20.0         | 57 KB     |            |   0% 


#### Use geopy library to get the latitude and longitude values of Riyadh City.

In [172]:
address = 'Riyadh, KSA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Riyadh City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Riyadh City are 24.7471687, 46.7623279.


In [56]:
map_riyadh = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_riyadh)  
    
map_riyadh

Let's choose the neighborhoods in North Riyadh

we will select neighborhoods which are in the box (lat_min = 24.68, long_min = 46.60, lat_max = 24.94, long_max = 46.8)

In [111]:
lat_min = 24.68
long_min = 46.60
lat_max = 24.94
long_max = 46.8

neighborhoods_north = pd.DataFrame(neighborhoods.loc[(neighborhoods["Longitude"]>long_min) &\
            (neighborhoods["Longitude"]<long_max) &\
            (neighborhoods["Latitude"]>lat_min) &\
            (neighborhoods["Latitude"]<lat_max)]).reset_index(drop=True)

neighborhoods_north

,Neighborhood,Longitude,Latitude
0,East Umm Al Hamam Dist.,$46.65804,$24.68646
1,North Mathar Dist.,$46.66676,$24.69113
2,West Umm Al Hamam Dist.,$46.64453,$24.68997
3,Al Rahmaniyah Dist.,$46.65934,$24.71613
4,Al Shohda Dist.,$46.73543,$24.78817
5,King Fahd Dist.,$46.67036,$24.74062
6,Al Falah Dist.,$46.70920,$24.79702
7,Al Nada Dist.,$46.68297,$24.80584
8,Al Mursalat Dist.,$46.68991,$24.74886
9,Al Nuzha Dist.,$46.70789,$24.75643


In [101]:
print('We choosed {} neighborhoods in  North Riyadh City'.format(neighborhoods_north.shape[0]))

We choosed 54 neighborhoods in  North Riyadh City


Lets's plot these neighborhoods on a folium map

In [102]:
map_riyadh_north = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, neighborhood in zip(neighborhoods_north['Latitude'], neighborhoods_north['Longitude'], neighborhoods_north['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_riyadh_north)  
    
map_riyadh_north

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'L2YGMVEQ4LJAC5K1I3EIGVOS4JCDH3CBVFJTLZLG34IQLRXB' # your Foursquare ID
CLIENT_SECRET = 'YGMFHYRSQHRA2ZEQMQWVL2IJ2FNC15VOVRKTYDFIDK4DXJSB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L2YGMVEQ4LJAC5K1I3EIGVOS4JCDH3CBVFJTLZLG34IQLRXB
CLIENT_SECRET:YGMFHYRSQHRA2ZEQMQWVL2IJ2FNC15VOVRKTYDFIDK4DXJSB


#### Let's explore Al Sulaimaniyah neighborhood in our dataframe.

Get the neighborhood's name.

In [112]:
neighborhoods_north.loc[18,'Neighborhood']

'Al Sulaimaniyah Dist.'

In [114]:
neighborhood_latitude = neighborhoods_north.loc[18, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods_north.loc[18, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods_north.loc[18, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Al Sulaimaniyah Dist. are 24.700762485390896, 46.7023448861621.


#### Now, let's get the top 100 venues that are in Al Sulaimaniyah Dist within a radius of 1000 meters.

First, let's create the GET request URL. Name your URL **url**.

In [115]:
radius=1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, 
                                                                                                                            neighborhood_longitude,radius, LIMIT)
# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=L2YGMVEQ4LJAC5K1I3EIGVOS4JCDH3CBVFJTLZLG34IQLRXB&client_secret=YGMFHYRSQHRA2ZEQMQWVL2IJ2FNC15VOVRKTYDFIDK4DXJSB&v=20180605&ll=24.700762485390896,46.7023448861621&radius=1000&limit=100'

In [116]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d00f20f6a6071550afd52a6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'As-Sulaimanyah',
  'headerFullLocation': 'As-Sulaimanyah, Riyadh',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 147,
  'suggestedBounds': {'ne': {'lat': 24.709762494390905,
    'lng': 46.71223281030234},
   'sw': {'lat': 24.69176247639089, 'lng': 46.692456962021865}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53f4ac57498e3bd97f602bb3',
       'name': 'Nozomi (نوزومي)',
       'location': {'address': "Abdulaziz Bin Musa'ed Bin Jalawi St",
        'crossStreet': 'at Sultan Bin Abdulaziz St',
        'lat': 24.700110996630595,
        'lng': 46.704

There are 147 venues in this neighborhood.

Let's limit our search to restarants only. we will query the Food section : section=food

In [121]:
url_food = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, 
                                                                                                                            neighborhood_longitude,radius, LIMIT)
# display URL
url_food


'https://api.foursquare.com/v2/venues/explore?&client_id=L2YGMVEQ4LJAC5K1I3EIGVOS4JCDH3CBVFJTLZLG34IQLRXB&client_secret=YGMFHYRSQHRA2ZEQMQWVL2IJ2FNC15VOVRKTYDFIDK4DXJSB&v=20180605&ll=24.700762485390896,46.7023448861621&radius=1000&limit=100&section=food'

In [122]:
results_food = requests.get(url_food).json()
results_food

{'meta': {'code': 200, 'requestId': '5d00f56fdd57972e79615c63'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'As-Sulaimanyah',
  'headerFullLocation': 'As-Sulaimanyah, Riyadh',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 123,
  'suggestedBounds': {'ne': {'lat': 24.709762494390905,
    'lng': 46.71223281030234},
   'sw': {'lat': 24.69176247639089, 'lng': 46.692456962021865}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53f4ac57498e3bd97f602bb3',
       'name': 'Nozomi (نوزومي)',
       'location': {'address': "Abdulaziz Bin Musa'ed Bin Jalawi St",
        'crossStreet': 'at Sultan Bin Abdulaziz St',
        'lat': 24.700110996630595,
  

In [123]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [124]:
venues = results_food['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Nozomi (نوزومي),Japanese Restaurant,$24.70011,$46.70471
1,Crust Corner (ركن كرست),Restaurant,$24.70075,$46.70208
2,Maharaja ِEast by Vineet (مهراجا),Indian Restaurant,$24.70227,$46.70142
3,Draft Café (درافت كافيه),Café,$24.70071,$46.70207
4,San Carlo Cicchetti,Italian Restaurant,$24.70125,$46.70526
5,Lamis (لميس),Middle Eastern Restaurant,$24.70016,$46.70252
6,Piedays (پايدايز),Café,$24.70067,$46.70214
7,Eataly (إيتالي),Italian Restaurant,$24.70409,$46.70283
8,Maestro Pizza (مايسترو بيتزا),Pizza Place,$24.70059,$46.70468
9,Zaatar w Zeit (زعتر و زيت),Breakfast Spot,$24.70302,$46.69902


And how many venues were returned by Foursquare?

In [119]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [150]:
neighborhood_venues = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhood_venues)  
    
neighborhood_venues

In [158]:
nearby_venues_aggr = nearby_venues[['categories','name']].groupby(['categories'])['name'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) #\
#                             .head(5)
nearby_venues_aggr

,categories,count
6,Café,18
5,Burger Joint,10
18,Middle Eastern Restaurant,9
21,Restaurant,8
20,Pizza Place,7
4,Breakfast Spot,5
17,Lebanese Restaurant,5
13,Indian Restaurant,4
14,Italian Restaurant,4
15,Japanese Restaurant,4


#### Let's create a function to repeat the same process to all the neighborhoods in North Riyadh

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [229]:
north_riyadh_venues = getNearbyVenues(names=neighborhoods_north['Neighborhood'],
                                   latitudes=neighborhoods_north['Latitude'],
                                   longitudes=neighborhoods_north['Longitude']
                                  )

East Umm Al Hamam Dist.
North Mathar Dist.
West Umm Al Hamam Dist.
Al Rahmaniyah Dist.
Al Shohda Dist.
King Fahd Dist.
Al Falah Dist.
Al Nada Dist.
Al Mursalat Dist.
Al Nuzha Dist.
Al Woroud Dist.
King Faisal Dist.
King Abdullah Dist.
Al Wahah Dist.
Salahuddin Dist.
King Abdulaziz Dist.
Al Rabwah Dist.
Al Muhammadiyah Dist.
Al Sulaimaniyah Dist.
Al Olaya Dist.
Al Manar Dist.
Al Quds Dist.
Al Wadi Dist.
Al Nafel Dist.
Al Maseef Dist.
Al Taawun Dist.
Al Ezdihar Dist.
Al Andalus Dist.
Al Rawdah Dist.
Al Rawabi Dist.
Al Rayan Dist.
Qurtubah Dist.
Al Rabie Dist.
Al Mughrazat Dist.
Al Aqeeq Dist.
Al Nakheel Dist.
Al Ghadeer Dist.
Al Muruj Dist.
Al Sahafah Dist.
Al Raed Dist.
Al Hamra Dist.
Al Zahra Dist.
Ishbiliyah Dist.
Al Yarmuk Dist.
Al Munisiyah Dist.
Al Khuzama Dist.
Hitteen Dist.
Al Yasmeen Dist.
Al Arid Dist.
King Khalid International Airport
Al Narjis Dist.
Imam Muhammed Bin Saud Islamic University
Ghirnatah Dist.
King Saud University


#### Let's check the size of the resulting dataframe

In [230]:
print(north_riyadh_venues.shape)
north_riyadh_venues.head()

(1071, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Mathar Dist.,$24.69113,$46.66676,Walking Track - Prince Turki Bin Abdulaziz AlA...,$24.68956,$46.66768,Track
1,North Mathar Dist.,$24.69113,$46.66676,Danube (الدانوب),$24.69243,$46.66880,Supermarket
2,North Mathar Dist.,$24.69113,$46.66676,Pandora (باندورا),$24.69202,$46.66876,Jewelry Store
3,North Mathar Dist.,$24.69113,$46.66676,مجوهرات الكوهجي Kooheji Jewellery,$24.69184,$46.66877,Jewelry Store
4,North Mathar Dist.,$24.69113,$46.66676,Aura (اورا),$24.69155,$46.66911,Furniture / Home Store


Let's check how many venues were returned for each neighborhood

In [175]:
north_riyadh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Al Andalus Dist.,10,10,10,10,10,10
Al Aqeeq Dist.,13,13,13,13,13,13
Al Arid Dist.,8,8,8,8,8,8
Al Ezdihar Dist.,15,15,15,15,15,15
Al Falah Dist.,10,10,10,10,10,10
Al Ghadeer Dist.,16,16,16,16,16,16
Al Hamra Dist.,25,25,25,25,25,25
Al Khuzama Dist.,4,4,4,4,4,4
Al Manar Dist.,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [176]:
print('There are {} uniques categories.'.format(len(north_riyadh_venues['Venue Category'].unique())))

There are 167 uniques categories.


### Analyze Each Neighborhood

In [177]:
# one hot encoding
north_riyadh_onehot = pd.get_dummies(north_riyadh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_riyadh_onehot['Neighborhood'] = north_riyadh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_riyadh_onehot.columns[-1]] + list(north_riyadh_onehot.columns[:-1])
north_riyadh_onehot = north_riyadh_onehot[fixed_columns]

north_riyadh_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Butcher,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gift Shop,Go Kart Track,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hookah Bar,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Latin American Restaurant,Lawyer,Leather Goods Store,Lebanese Restaurant,Lounge,Luggage Store,Market,Massage Studio,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Mountain,National Park,New American Restaurant,Noodle House,Optical Shop,Other Repair Shop,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pop-Up Shop,Public Art,Rental Car Location,Resort,Rest Area,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taxi Stand,Tea Room,Tennis Court,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track,Trail,Turkish Restaurant,Video Game Store,Watch Shop,Women's Store
0,North Mathar Dist.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,North Mathar Dist.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,North Mathar Dist.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,North Mathar Dist.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,North Mathar Dist.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [178]:
north_riyadh_onehot.shape

(1070, 168)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [179]:
north_riyadh_grouped = north_riyadh_onehot.groupby('Neighborhood').mean().reset_index()
north_riyadh_grouped.head(10)

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Butcher,Cafeteria,Café,Camera Store,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gift Shop,Go Kart Track,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hookah Bar,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Latin American Restaurant,Lawyer,Leather Goods Store,Lebanese Restaurant,Lounge,Luggage Store,Market,Massage Studio,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Mountain,National Park,New American Restaurant,Noodle House,Optical Shop,Other Repair Shop,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pop-Up Shop,Public Art,Rental Car Location,Resort,Rest Area,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shawarma Place,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taxi Stand,Tea Room,Tennis Court,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track,Trail,Turkish Restaurant,Video Game Store,Watch Shop,Women's Store
0,Al Andalus Dist.,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.10000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000
1,Al Aqeeq Dist.,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.00000,$0.07692,$0.00000,$

#### Let's confirm the new size

In [180]:
north_riyadh_grouped.shape

(53, 168)

#### Let's print each neighborhood along with the top 5 most common venues

In [181]:
num_top_venues = 5

for hood in north_riyadh_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_riyadh_grouped[north_riyadh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Al Andalus Dist.----
          venue     freq
0  Dessert Shop $0.10000
1      Tea Room $0.10000
2    Donut Shop $0.10000
3      Pharmacy $0.10000
4   Candy Store $0.10000


----Al Aqeeq Dist.----
                       venue     freq
0                      Hotel $0.15000
1                Coffee Shop $0.15000
2        Arts & Crafts Store $0.08000
3  Middle Eastern Restaurant $0.08000
4             Shawarma Place $0.08000


----Al Arid Dist.----
                  venue     freq
0                Lounge $0.25000
1                  Farm $0.12000
2  Fast Food Restaurant $0.12000
3            Hookah Bar $0.12000
4    Country Dance Club $0.12000


----Al Ezdihar Dist.----
               venue     freq
0        Coffee Shop $0.13000
1     Breakfast Spot $0.13000
2  Convenience Store $0.07000
3             Market $0.07000
4             Bakery $0.07000


----Al Falah Dist.----
         venue     freq
0  Coffee Shop $0.30000
1        Hotel $0.20000
2   Donut Shop $0.20000
3      Theater $0.1000

          venue     freq
0          Park $0.33000
1  Dessert Shop $0.22000
2           Spa $0.11000
3        Market $0.11000
4      Gym Pool $0.11000


----King Abdulaziz Dist.----
                  venue     freq
0      Airport Terminal $1.00000
1                   ATM $0.00000
2  Outdoor Supply Store $0.00000
3     Mobile Phone Shop $0.00000
4                 Motel $0.00000


----King Abdullah Dist.----
         venue     freq
0        Hotel $0.25000
1       Hostel $0.25000
2  Flower Shop $0.25000
3  Coffee Shop $0.25000
4    Pet Store $0.00000


----King Fahd Dist.----
                       venue     freq
0  Middle Eastern Restaurant $0.18000
1             Breakfast Spot $0.14000
2                   Pharmacy $0.07000
3                  Juice Bar $0.07000
4                Flower Shop $0.04000


----King Faisal Dist.----
                       venue     freq
0  Middle Eastern Restaurant $0.25000
1                   Pie Shop $0.12000
2             Sandwich Place $0.12000
3       Fast 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [182]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [249]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_riyadh_grouped['Neighborhood']

for ind in np.arange(north_riyadh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_riyadh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Andalus Dist.,Pharmacy,Gift Shop,Middle Eastern Restaurant,Food Truck,Candy Store,Breakfast Spot,Tea Room,Juice Bar,Donut Shop,Dessert Shop
1,Al Aqeeq Dist.,Hotel,Coffee Shop,Tailor Shop,Arts & Crafts Store,Shawarma Place,Café,Fast Food Restaurant,Falafel Restaurant,Health Food Store,Middle Eastern Restaurant
2,Al Arid Dist.,Lounge,Hookah Bar,Country Dance Club,Bar,Fast Food Restaurant,Rest Area,Farm,Falafel Restaurant,Flower Shop,Fish Market
3,Al Ezdihar Dist.,Coffee Shop,Breakfast Spot,Pharmacy,Fast Food Restaurant,Restaurant,Bakery,Falafel Restaurant,Supermarket,Public Art,Market
4,Al Falah Dist.,Coffee Shop,Hotel,Donut Shop,Burger Joint,Theater,Food Court,Dumpling Restaurant,Eastern European Restaurant,Dog Run,Discount Store


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 7 clusters.

In [250]:
# set number of clusters
kclusters = 7

north_riyadh_grouped_clustering = north_riyadh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_riyadh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 4, 2, 0, 2, 2, 0, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [251]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_riyadh_merged =neighborhoods_north 
north_riyadh_merged = neighborhoods_north.drop(neighborhoods_north[(neighborhoods_north.Neighborhood == "East Umm Al Hamam Dist.")].index)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_riyadh_merged = north_riyadh_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_riyadh_merged.head() # check the last columns!

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North Mathar Dist.,$46.66676,$24.69113,2,Jewelry Store,Coffee Shop,Clothing Store,Watch Shop,Dessert Shop,Bakery,Ice Cream Shop,Accessories Store,Cosmetics Shop,Electronics Store
2,West Umm Al Hamam Dist.,$46.64453,$24.68997,2,Falafel Restaurant,Breakfast Spot,Middle Eastern Restaurant,Mobile Phone Shop,Supermarket,BBQ Joint,Restaurant,Dessert Shop,Pizza Place,Eastern European Restaurant
3,Al Rahmaniyah Dist.,$46.65934,$24.71613,2,Dessert Shop,Chocolate Shop,Coffee Shop,Pharmacy,Optical Shop,Mobile Phone Shop,Flower Shop,Bar,Cupcake Shop,Burger Joint
4,Al Shohda Dist.,$46.73543,$24.78817,0,Hotel,Café,National Park,Dessert Shop,Market,Food Truck,Motel,Bakery,Coffee Shop,Gym
5,King Fahd Dist.,$46.67036,$24.74062,2,Middle Eastern Restaurant,Breakfast Spot,Pharmacy,Juice Bar,Massage Studio,Market,Flower Shop,Sandwich Place,Café,Candy Store


Finally, let's visualize the resulting clusters

In [252]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_riyadh_merged['Latitude'], north_riyadh_merged['Longitude'], north_riyadh_merged['Neighborhood'], north_riyadh_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [270]:
cluster1 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 0, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster1 has {} neighborhoods.'.format(cluster1.shape[0]))

Cluster1 has 14 neighborhoods.


In [271]:
cluster1

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Al Shohda Dist.,$24.78817,0,Hotel,Café,National Park,Dessert Shop,Market,Food Truck,Motel,Bakery,Coffee Shop,Gym
6,Al Falah Dist.,$24.79702,0,Coffee Shop,Hotel,Donut Shop,Burger Joint,Theater,Food Court,Dumpling Restaurant,Eastern European Restaurant,Dog Run,Discount Store
7,Al Nada Dist.,$24.80584,0,Women's Store,Park,Coffee Shop,Food Truck,Discount Store,Farm,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
12,King Abdullah Dist.,$24.73323,0,Hotel,Coffee Shop,Flower Shop,Hostel,Airport Terminal,Farm,Food Court,Food & Drink Shop,Accessories Store,Fish Market
21,Al Quds Dist.,$24.75480,0,Coffee Shop,Gift Shop,Video Game Store,Middle Eastern Restaurant,Fast Food Restaurant,Indian Restaurant,Grocery Store,Restaurant,Breakfast Spot,Supermarket
23,Al Nafel Dist.,$24.78188,0,Grocery Store,Hotel,Athletics & Sports,Gym,Motel,Park,Hostel,Coffee Shop,Café,Hookah Bar
29,Al Rawabi Dist.,$24.69527,0,Bakery,Supermarket,Deli / Bodega,Fruit & Vegetable Store,Market,Middle Eastern Restaurant,Dessert Shop,Food Truck,Farmers Market,Tailor Shop
32,Al Rabie Dist.,$24.79547,0,Health Food Store,Coffee Shop,Market,Mobile Phone Shop,Food Truck,Mountain,Juice Bar,Diner,Paper / Office Supplies Store,Pizza Place
33,Al Mughrazat Dist.,$24.76412,0,Coffee Shop,Food Truck,Bakery,Motel,Burger Joint,Trail,Italian Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Gym
34,Al Aqeeq Dist.,$24.77362,0,Hotel,Coffee Shop,Tailor Shop,Arts & Crafts Store,Shawarma Place,Café,Fast Food Restaurant,Falafel Restaurant,Health Food Store,Middle Eastern Restaurant


#### Cluster 2

In [275]:
cluster2 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 1, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster2 has {} neighborhoods.'.format(cluster2.shape[0]))

Cluster2 has 1 neighborhoods.


In [274]:
cluster2

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,King Khalid International Airport,$24.92175,1,Hotel,Food Service,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 3

In [276]:
cluster3 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 2, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster3 has {} neighborhoods.'.format(cluster3.shape[0]))

Cluster3 has 31 neighborhoods.


In [277]:
cluster3

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North Mathar Dist.,$24.69113,2,Jewelry Store,Coffee Shop,Clothing Store,Watch Shop,Dessert Shop,Bakery,Ice Cream Shop,Accessories Store,Cosmetics Shop,Electronics Store
2,West Umm Al Hamam Dist.,$24.68997,2,Falafel Restaurant,Breakfast Spot,Middle Eastern Restaurant,Mobile Phone Shop,Supermarket,BBQ Joint,Restaurant,Dessert Shop,Pizza Place,Eastern European Restaurant
3,Al Rahmaniyah Dist.,$24.71613,2,Dessert Shop,Chocolate Shop,Coffee Shop,Pharmacy,Optical Shop,Mobile Phone Shop,Flower Shop,Bar,Cupcake Shop,Burger Joint
5,King Fahd Dist.,$24.74062,2,Middle Eastern Restaurant,Breakfast Spot,Pharmacy,Juice Bar,Massage Studio,Market,Flower Shop,Sandwich Place,Café,Candy Store
9,Al Nuzha Dist.,$24.75643,2,Juice Bar,Coffee Shop,Hotel,Flower Shop,Trail,Burger Joint,Shawarma Place,Breakfast Spot,Dessert Shop,Soccer Field
10,Al Woroud Dist.,$24.72445,2,Café,Middle Eastern Restaurant,Coffee Shop,Gift Shop,Pharmacy,Other Repair Shop,Pizza Place,Breakfast Spot,Salon / Barbershop,Camera Store
11,King Faisal Dist.,$24.76186,2,Middle Eastern Restaurant,Pharmacy,Pie Shop,Fast Food Restaurant,Ice Cream Shop,Sandwich Place,Restaurant,Electronics Store,Farmers Market,Farm
14,Salahuddin Dist.,$24.73338,2,Café,Sandwich Place,Coffee Shop,Health Food Store,Juice Bar,Breakfast Spot,Restaurant,Dessert Shop,Comic Shop,Mobile Phone Shop
16,Al Rabwah Dist.,$24.69330,2,Shawarma Place,Pizza Place,Dessert Shop,Park,Bakery,Juice Bar,Café,Burger Joint,Shopping Plaza,Lounge
17,Al Muhammadiyah Dist.,$24.73258,2,Ice Cream Shop,Bakery,Café,Boutique,Coffee Shop,Plaza,Chocolate Shop,Cupcake Shop,Pop-Up Shop,Sandwich Place


#### Cluster 4

In [278]:
cluster4 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 3, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster4 has {} neighborhoods.'.format(cluster4.shape[0]))

Cluster4 has 1 neighborhoods.


In [279]:
cluster4

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,King Abdulaziz Dist.,$24.71816,3,Airport Terminal,Women's Store,Event Service,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 5

In [280]:
cluster5 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 4, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster5 has {} neighborhoods.'.format(cluster5.shape[0]))

Cluster5 has 3 neighborhoods.


In [281]:
cluster5

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Al Wahah Dist.,$24.74109,4,Food Truck,Hookah Bar,Lounge,Gym / Fitness Center,Event Service,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
44,Al Munisiyah Dist.,$24.83215,4,Farm,Hookah Bar,Market,Lounge,Rental Car Location,Resort,Breakfast Spot,Falafel Restaurant,Women's Store,Flower Shop
48,Al Arid Dist.,$24.89791,4,Lounge,Hookah Bar,Country Dance Club,Bar,Fast Food Restaurant,Rest Area,Farm,Falafel Restaurant,Flower Shop,Fish Market


#### Cluster 6

In [282]:
cluster6 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 5, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster6 has {} neighborhoods.'.format(cluster6.shape[0]))

Cluster6 has 1 neighborhoods.


In [283]:
cluster6

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Al Narjis Dist.,$24.88684,5,Hookah Bar,Women's Store,Event Service,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 7

In [284]:
cluster7 = north_riyadh_merged.loc[north_riyadh_merged['Cluster Labels'] == 6, north_riyadh_merged.columns[[0] + list(range(2, north_riyadh_merged.shape[1]))]]
print('Cluster7 has {} neighborhoods.'.format(cluster7.shape[0]))

Cluster7 has 2 neighborhoods.


In [285]:
cluster7

,Neighborhood,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Al Mursalat Dist.,$24.74886,6,Café,Soccer Field,Shopping Mall,Gym Pool,Donut Shop,Mobile Phone Shop,Electronics Store,Athletics & Sports,Farmers Market,Flower Shop
51,Imam Muhammed Bin Saud Islamic University,$24.81446,6,Café,Food Court,Fast Food Restaurant,Falafel Restaurant,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Farmers Market,Farm
